## Задание

Мы - онлайн магазин спортивных товаров: одежды, обуви, аксессуаров и спорт-питания. На главной странице магазина мы показываем пользователям баннеры с целью стимулирования своих продаж. Сейчас там случайно показывается один из 5 баннеров. Каждый баннер рекламирует конкретный продукт или всю компанию. Наши маркетологи считают, что опыт взаимодействия с баннерами может различаться по сегментам, а их эффективность - зависеть от особенностей пользовательского поведения.

У менеджера компании появилось предложение от партнеров продать это место для баннера и рекламировать там другой сервис (оплата предполагается по CPC-модели).

Помогите менеджеру принять решение.

## Подключение библиотек и скриптов

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [2]:
plt.style.use('seaborn-bright')
plt.rcParams['figure.figsize'] = (6, 4)
matplotlib.rcParams.update({'font.size': 14})

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 50)

In [4]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#### Пути к директориям и файлам

In [5]:
DATASET_PATH = '../test_data/ha_data.csv'

## Загрузка данных

Описание датасета: у нас есть информация о том, какой баннер показался пользователю, кликнул ли он на него, а так же информация о покупках пользователей.

∙ **title** - тип события (показ, клик или покупка)  
∙ **user** - уникальный идентификатор клиента  
∙ **product** - продукт баннера/покупки  
∙ **page_id** - уникальный номер страницы для связки событий (NA для покупок)  
∙ **order_id** - уникальный номер покупки (NA для кликов и показов баннера)  
∙ **time** - время совершения действия  
∙ **site_version** - версия сайта (мобильная или десктопная)  

In [6]:
#!cat ../test_data/ha_data.csv | head -10

In [7]:
df = pd.read_csv(DATASET_PATH, sep=';', dtype={'order_id': str, 'page_id': str})
df.head()

,order_id,page_id,product,site_version,time,title,user
0,NaN,3.0,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,NaN,3699687.0,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,NaN,14.0,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,NaN,10362176.0,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,NaN,14.0,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1


#### Приведение типов

In [8]:
df['time'] = pd.to_datetime(df['time'])

## Общая информации о данных

In [9]:
df.head()

,order_id,page_id,product,site_version,time,title,user
0,NaN,3.0,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,NaN,3699687.0,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,NaN,14.0,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,NaN,10362176.0,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,NaN,14.0,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1


In [10]:
df.shape

(8471226, 7)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8471226 entries, 0 to 8471225
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   order_id      object        
 1   page_id       object        
 2   product       object        
 3   site_version  object        
 4   time          datetime64[ns]
 5   title         object        
 6   user          object        
dtypes: datetime64[ns](1), object(6)
memory usage: 452.4+ MB


In [12]:
df.nunique()

order_id         248722
page_id         7393319
product               5
site_version          2
time            5799553
title                 3
user            4254621
dtype: int64

In [13]:
df.isna().sum()

order_id        8222504
page_id          248722
product               0
site_version          0
time                  0
title                 0
user                  0
dtype: int64

## Обзор данных

#### Категориальные переменные

In [14]:
for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '-' * 100 + '\n')

order_id

26486.0     1
243693.0    1
356858.0    1
503053.0    1
383354.0    1
           ..
4891.0      1
178729.0    1
817016.0    1
198784.0    1
240132.0    1
Name: order_id, Length: 248722, dtype: int64
----------------------------------------------------------------------------------------------------

page_id

15672877.0    2
7540809.0     2
21336242.0    2
24697098.0    2
11144875.0    2
             ..
23055531.0    1
17230383.0    1
4410359.0     1
4361163.0     1
24019492.0    1
Name: page_id, Length: 7393319, dtype: int64
----------------------------------------------------------------------------------------------------

product

clothes             1786438
company             1725059
sneakers            1703345
sports_nutrition    1634625
accessories         1621759
Name: product, dtype: int64
----------------------------------------------------------------------------------------------------

site_version

mobile     6088340
desktop    2382886
Name: site_version, dtype:

Как мы видим **page_id** состоит из уникальных значений, так как каждому событию (клик/просмотр) присваивается неповторящийся номер.  
Но эти данные дублируют обратный параметр **order_id**, который присваивается каждой покупке. 

Я удалю параметр **page_id**, мы будем работать с **order_id**.

In [15]:
df.drop("page_id", axis=1, inplace=True)
df

,order_id,product,site_version,time,title,user
0,NaN,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,NaN,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,NaN,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,NaN,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,NaN,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1
...,...,...,...,...,...,...
8471221,NaN,accessories,desktop,2017-05-23 14:07:00,banner_show,user_4254616
8471222,NaN,clothes,mobile,2017-05-28 08:10:20,banner_show,user_4254617
8471223,NaN,sports_nutrition,mobile,2017-05-20 09:20:50,banner_show,user_4254618
8471224,NaN,sneakers,mobile,2017-05-28 19:25:42,banner_show,user_4254619


## Нулевые гипотезы

**Гипотеза 1**: Я верю, что опыт взаимодействия с баннерами может различаться по сегментам баннеров. Чтобы проверить это, я проанализирую каждый сегмент баннеров в разрезе пользовательских действий: покупки, кликов, просмотров;  


**Гипотеза 2**: Я верю, что эффективность баннеров равнозначна на мобильной и декстопной версии. Чтобы проверить это, я проанализирую конверсию в мобильной и декстопной версии;


**Гипотеза 3**: Если мы будем показывать товарные баннеры, а не с брендинговые, то количество продаж не изменится;  


**Гипотеза 4**: Если мы продадим баннерное место под CPC-рекламу, это не повлияет на наш доход;

### Конверсии

**Conversion Rate** = (число сконвертированных / общее число посетителей) * 100%

**CR** - коэффициент конверсии. Конверсией в интернет-маркетинге называют количественное соотношение пользователей ресурса, которые совершили целевое действие к общему числу посетителей. 

**CTR** - показатель кликабельности. Метрика в интернет-маркетинге. CTR определяется как отношение числа кликов на баннер к числу показов, измеряется в процентах.

Считаем соотношения:  

- количество купивших / число всех пользователей,    
- количество кликнувших / число всех пользователей,
- количество кликов / число показов,
- число заказов / число показов,
- число заказов / число кликов, (вроде нет в этом смысла)
                  
в целом и в разрезе типа баннера и типа сайта

### Users 

Рассмотрим детально пользователей магазина

In [16]:
n_users = df['user'].unique().shape[0]
n_users

4254621

### Orders

Мы знаем количество заказов, совершенных пользователями, но иногда эти заказы совершали одни и те же пользователи в разное время. Поэтому давайте узнаем количество уникальных пользователей, совершивших покупки.

In [17]:
df.head()

,order_id,product,site_version,time,title,user
0,NaN,company,mobile,2017-02-09 20:24:04,banner_show,user_0
1,NaN,company,mobile,2017-02-07 10:03:07,banner_show,user_0
2,NaN,sneakers,mobile,2017-01-29 13:02:23,banner_show,user_1
3,NaN,company,mobile,2017-04-12 15:39:19,banner_show,user_1
4,NaN,sneakers,mobile,2017-01-29 13:04:42,banner_click,user_1


In [18]:
df['order_id'].unique().shape[0]

248723

In [19]:
n_users_with_order = df[pd.notna(df['order_id'])]['user'].unique().shape[0]
n_users_with_order

237866

**248722** - Общее количество заказов  
**237866** - Количество уникальных пользователей, совершивших покупки

### Banner clicks

In [20]:
n_banner_clicks = df[(df['title'] == 'banner_click')]['user'].unique().shape[0]
n_banner_clicks

742459

**829185** - Общее количество кликов  
**742459** - Количество уникальных пользователей, кликнувших на объявление 

### Banner shows

In [21]:
n_banner_shows = df[(df['title'] == 'banner_show')]['user'].unique().shape[0]
n_banner_shows

4254621

**7393319** - Общее количество просмотров баннера  
**4254621** - Количество уникальных пользователей, просмотревших объявление 

#### Заказы

Процент пользователей, совершивших заказ, от всего числа пользователей

In [22]:
percent_users_with_order = (n_users_with_order / n_users) * 100
percent_users_with_order

5.590768249392837

#### Клики

Процент пользователей, кликнувших на объявление, от всего числа пользователей

In [23]:
percent_users_banner_clicks = (n_banner_clicks / n_users) * 100
percent_users_banner_clicks

17.450649540816915

#### Просмотры баннеров

Процент пользователей, просмотревших объявление, от всего числа пользователей

In [24]:
percent_users_banner_shows = (n_users / n_banner_shows) * 100
percent_users_banner_shows

100.0

Получается, каждый пользователь хотя бы раз просмотрел баннер и дальше уже либо кликнул по нему либо совершил покупку

### Site version

Взглянем на пользователей в разрезе desktop и mobile

In [25]:
df.groupby(['site_version']).size()

site_version
desktop    2382886
mobile     6088340
dtype: int64

В таблице выше мы видим количество всех действий в мобильной версии и декстоп.  
Пользователи совершают действия охотнее в мобильной версии, чем на сайте.

Рассмотрим подробнее

In [26]:
n_users_by_desktop = df[(df['site_version'] == 'desktop')]['user'].unique().shape[0]
n_users_by_desktop 

1424772

In [27]:
n_users_by_mobile = df[(df['site_version'] == 'mobile')]['user'].unique().shape[0]
n_users_by_mobile

2857189

**2857189** - Пользователей мобильной версии  
**1424772** - Пользователей декстоп

#### Узнаем количество пользователей мобильной версии и десктоп  

In [28]:
sum_users_by_site = n_users_by_mobile + n_users_by_desktop 

**4281961 > 4254621** - мы видим, что пользователей по типу устройств больше, чем суммарное количество уникальных пользователей, значит некоторые пользователи совершали действия и на мобильном устройстве и декстоп

In [29]:
# Количество пользователей, совершивших действие на моб и декстоп
sum_users_by_site - n_users 

27340

#### Далее рассмотрим количество уникальных пользовательских действий в зависимости от устройства 

In [30]:
df_users_actions = df.groupby(['site_version', 'title', 'user']).size().reset_index()  \
      .groupby(['site_version', 'title']).size() \
      .reset_index(name='count')                                    \
      .sort_values(['title'], ascending=[False])                    \
      .reset_index(drop = True)

df_users_actions

,site_version,title,count
0,desktop,order,125404
1,mobile,order,112574
2,desktop,banner_show,1424772
3,mobile,banner_show,2857189
4,desktop,banner_click,111334
5,mobile,banner_click,631645


In [31]:
df_users_actions['users_per_site_version'] = n_users_by_mobile

In [32]:
df_users_actions.loc[df_users_actions['site_version'].isin(['desktop']), 'users_per_site_version'] = n_users_by_desktop

In [33]:
df_users_actions

,site_version,title,count,users_per_site_version
0,desktop,order,125404,1424772
1,mobile,order,112574,2857189
2,desktop,banner_show,1424772,1424772
3,mobile,banner_show,2857189,2857189
4,desktop,banner_click,111334,1424772
5,mobile,banner_click,631645,2857189


In [34]:
df_users_actions['conversion_rate'] = (df_users_actions['count'] / df_users_actions['users_per_site_version']) * 100

#### Считаем конверсию по каждому действию в мобильной версии и декстоп

In [35]:
df_users_actions.sort_values(['title'], ascending=[False])\
    [['site_version', 'title','conversion_rate']].reset_index(drop = True)

,site_version,title,conversion_rate
0,desktop,order,8.80
1,mobile,order,3.94
2,desktop,banner_show,100.00
3,mobile,banner_show,100.00
4,desktop,banner_click,7.81
5,mobile,banner_click,22.11


Так как пользователь в любом случае просмотрел баннер и только потом совершил какое - либо действие, то конверсия по **banner_show** равна 100 и мы можем удалить эти строки

In [36]:
df_users_actions = df_users_actions.drop([2, 3])
df_users_actions

,site_version,title,count,users_per_site_version,conversion_rate
0,desktop,order,125404,1424772,8.80
1,mobile,order,112574,2857189,3.94
4,desktop,banner_click,111334,1424772,7.81
5,mobile,banner_click,631645,2857189,22.11


In [37]:
df_users_actions[['site_version', 'title', 'conversion_rate']]

,site_version,title,conversion_rate
0,desktop,order,8.80
1,mobile,order,3.94
4,desktop,banner_click,7.81
5,mobile,banner_click,22.11


### Product Type

**Давайте поближе рассмотрим какие действия совершают пользователи магазина, исходя из типа продукта на баннере**

In [38]:
df_product_type = df.groupby(['product', 'title', 'user']).size().reset_index()  \
        .groupby(['product', 'title']).size()                                    \
        .reset_index(name='count')                                        \
        .sort_values(['product'], ascending=[False])                             \
        .reset_index(drop = True)

df_product_type 

,product,title,count
0,sports_nutrition,banner_click,139854
1,sports_nutrition,banner_show,1173163
2,sports_nutrition,order,23609
3,sneakers,banner_click,174802
4,sneakers,banner_show,1163808
5,sneakers,order,66917
6,company,banner_click,140683
7,company,banner_show,1256721
8,clothes,banner_click,210109
9,clothes,banner_show,1164914


Из сводки выше мы можем заметить, что пользователи **не совершают покупки, когда видят баннеры о компании, а не товарные**

Разделим уникальные данные по каждому типу баннеров и пользовательских действий на общее число уникальных пользователей, чтобы посчитать конверсию

In [39]:
df_product_type['conversion_rate'] = (df_product_type['count'] / n_users) * 100

In [40]:
df_product_type.sort_values(['conversion_rate'], ascending=[False]).reset_index(drop = True)

,product,title,count,conversion_rate
0,company,banner_show,1256721,29.54
1,sports_nutrition,banner_show,1173163,27.57
2,clothes,banner_show,1164914,27.38
3,sneakers,banner_show,1163808,27.35
4,accessories,banner_show,1155775,27.17
5,clothes,banner_click,210109,4.94
6,sneakers,banner_click,174802,4.11
7,company,banner_click,140683,3.31
8,sports_nutrition,banner_click,139854,3.29
9,accessories,banner_click,131998,3.10


Мы видим, что параметр **banner_show** почти везде одинаковый, но баннеры с логотипом компании просматривают чаще. 
В любом случае, эти просмотры не приводят нас к продажам товара.

#### Orders per banner show

In [41]:
(110347 / 1164914) * 100 # clothes

9.472544754376717

In [42]:
(66917 / 1163808) * 100 # sneakers

5.749831587340867

In [43]:
(44160 / 1155775) * 100 # accessories

3.820812874478164

In [44]:
(23609 / 1173163) * 100 # sports_nutrition 

2.0124228261545927

Удалим значения banner_show

In [45]:
df_product_type = df_product_type.sort_values(['conversion_rate'], ascending=[False]).reset_index(drop = True)[5:]
df_product_type

,product,title,count,conversion_rate
5,clothes,banner_click,210109,4.94
6,sneakers,banner_click,174802,4.11
7,company,banner_click,140683,3.31
8,sports_nutrition,banner_click,139854,3.29
9,accessories,banner_click,131998,3.10
10,clothes,order,110347,2.59
11,sneakers,order,66917,1.57
12,accessories,order,44160,1.04
13,sports_nutrition,order,23609,0.55


Из таблицы видно, что пользователи совершают больше всего покупок, когда видят рекламу позиций из категории **clothes**

Также по типу действия **banner_click** лидируют баннеры типа **clothes** и **sneakers**, но мы можем заметить, что все остальные категории баннеров вызывают у пользователей примерно одинаковый интерес. 

#### Orders per clicks

In [46]:
(110347 / 210109) * 100 # clothes

52.518930650281526

In [47]:
(66917 / 174802) * 100 # sheakers

38.28159860871157

In [48]:
(44160 / 131998) * 100 # accessories

33.45505234927802

In [49]:
(23609 / 139854) * 100 # sports_nutrition

16.88117608363007

### Расчеты

Если средняя стоимость товара - N рублей, а партнер предлагает продать место по CPC, то можно проверить выгодно ли нам продавать через следующую функцию:

In [8]:
def check_income(click_price, order_price, convertion_click, conversion_price):
    a = click_price * convertion_click
    b = (order_price * conversion_price) 
    
    print(f'{a} - Предполагаемый доход CPC. {b} - Предполагаемый доход от продажи.')
    if a < b:
        print(f'{a} - Отдавать баннерное место под CPC невыгодно')
    else:
        (f'{b} - Отдавать баннерное место под CPC выгодно')

Предположим, наш средний чек равен 5000, а средняя стоимость за клик за медийную рекламу составляет (баннеры) - 43 рубля по курсу на февраль 2021. Возьмем это значение.

In [7]:
check_income(43, 5000, 17.45, 5.59) # 

750.35 - предполагаемый доход CPC. 27950.0 - предполагаемый доход от продажи.
750.35 - Отдавать баннерное место под CPC невыгодно


Мы можем использовать эту формулу под каждую категорию в зависимости от ее цены и стоимости клика

### Выводы и рекомендации

1) Мы заметили, что пользователи ни разу не покупали с брендинговых товарных баннеров - значит вместо товарных баннеров мы можем отдать место под CPC 


2) Заказов совершается значительно больше с десктоп версии сайта в отличии от мобильной. Конверсия в 8.80% и 3.94% соответственно. Но кликают охотнее в мобильной версии сайта - 22.11% по сравнению с 7.81%. Думаю, мы можем отдать место под CPC именно на мобильной версии

3) По типу продукта - меньше всего совершают заказов с баннерами, изображающими аксессуары и спортивное питание. Обычно стоимость таких товаров меньше чем цена кроссовок и одежды, и, вероятно, нам также выгоднее продать это место партнеру.

### Time

Обратим внимание еще на некоторые аспекты:

Добавим новый столбец **день недели** и изучим поведение пользователей в разрезе будних дней и выходных.

In [50]:
df['day_of_week'] = df['time'].dt.day_name()

In [51]:
df['month'] = df['time'].dt.month_name()

Столбец **time** можно удалить за ненадобностью

In [52]:
df.drop("time", axis=1, inplace=True)

In [53]:
df.head()

,order_id,product,site_version,title,user,day_of_week,month
0,NaN,company,mobile,banner_show,user_0,Thursday,February
1,NaN,company,mobile,banner_show,user_0,Tuesday,February
2,NaN,sneakers,mobile,banner_show,user_1,Sunday,January
3,NaN,company,mobile,banner_show,user_1,Wednesday,April
4,NaN,sneakers,mobile,banner_click,user_1,Sunday,January


In [54]:
df['month'].value_counts()

May         2130108
April       1960088
March       1650715
January     1461098
February    1269217
Name: month, dtype: int64

In [55]:
df['day_of_week'].value_counts()

Tuesday      1228343
Monday       1228112
Sunday       1226920
Wednesday    1226023
Saturday     1194112
Friday       1188163
Thursday     1179553
Name: day_of_week, dtype: int64